In [ ]:
# Cell 1: Setup and Imports
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Add the 'src' directory to the path so we can import our custom code
sys.path.append(os.path.abspath(os.path.join('..')))

# Import our custom optimization logic from src/portfolio_optimizer.py
from src.portfolio_optimizer import optimize_portfolio, calculate_annualized_metrics

# Configure Plotting
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 8)

print("Setup Complete. Libraries Loaded.")

ImportError: cannot import name 'optimize_portfolio' from 'src.portfolio_optimizer' (/Users/abbygail/portfolio-optimization/src/portfolio_optimizer.py)

In [ ]:
# Cell 2: Load Data
# Load Cleaned Data from Task 1
df = pd.read_csv('../data/processed/cleaned_close_prices.csv', index_col=0, parse_dates=True)

# Calculate Daily Returns (Percentage Change)
# We drop the first row because it becomes NaN (cannot calculate change for day 0)
returns = df.pct_change().dropna()

print("Data Loaded. Shape:", returns.shape)
print("\nFirst 5 rows of Returns:")
print(returns.head())

In [ ]:
# Cell 3: Annualized Metrics
# We multiply by 252 because there are ~252 trading days in a year
mean_returns = returns.mean() * 252
cov_matrix = returns.cov() * 252

print("Expected Annual Returns:")
print(mean_returns)

print("\nCovariance Matrix (Risk Relationships):")
print(cov_matrix)

In [ ]:
# Cell 4: Monte Carlo Simulation
np.random.seed(42) # Set seed so results are reproducible
num_portfolios = 10000
results = np.zeros((3, num_portfolios)) # Array to store results (Vol, Ret, Sharpe)
weights_record = [] # List to store the specific weights for each portfolio

print(f"Simulating {num_portfolios} random portfolios...")

for i in range(num_portfolios):
    # 1. Generate random weights (that sum to 1.0)
    weights = np.random.random(len(mean_returns))
    weights /= np.sum(weights)
    weights_record.append(weights)
    
    # 2. Calculate Portfolio Return and Volatility
    portfolio_return = np.sum(mean_returns * weights)
    # The dot product calculation for variance
    portfolio_std_dev = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    
    # 3. Store results
    results[0,i] = portfolio_std_dev # Risk (X-axis)
    results[1,i] = portfolio_return  # Return (Y-axis)
    # Sharpe Ratio (assuming 0% risk-free rate for simplicity)
    results[2,i] = results[1,i] / results[0,i]

# Convert to DataFrame for easier analysis
results_frame = pd.DataFrame(results.T, columns=['Volatility', 'Return', 'Sharpe'])
print("Simulation Complete.")

In [ ]:
# Cell 5: Optimization Results

# 1. Locate the Max Sharpe Portfolio (Highest value in column 'Sharpe')
max_sharpe_idx = results_frame['Sharpe'].idxmax()
max_sharpe_port = results_frame.iloc[max_sharpe_idx]
max_sharpe_weights = weights_record[max_sharpe_idx]

# 2. Locate the Min Volatility Portfolio (Lowest value in column 'Volatility')
min_vol_idx = results_frame['Volatility'].idxmin()
min_vol_port = results_frame.iloc[min_vol_idx]
min_vol_weights = weights_record[min_vol_idx]

print("--- Optimal Portfolio (Max Sharpe Ratio) ---")
print(f"Return:     {max_sharpe_port['Return']:.2%}")
print(f"Volatility: {max_sharpe_port['Volatility']:.2%}")
print(f"Sharpe Ratio: {max_sharpe_port['Sharpe']:.2f}")
print("Weights:")
for i, asset in enumerate(mean_returns.index):
    print(f"  {asset}: {max_sharpe_weights[i]:.2%}")

print("\n--- Safest Portfolio (Min Volatility) ---")
print(f"Return:     {min_vol_port['Return']:.2%}")
print(f"Volatility: {min_vol_port['Volatility']:.2%}")
print("Weights:")
for i, asset in enumerate(mean_returns.index):
    print(f"  {asset}: {min_vol_weights[i]:.2%}")

In [ ]:
# Cell 6: Visualization
plt.figure(figsize=(12, 8))

# Scatter plot of all portfolios
# Color (c) is determined by Sharpe Ratio
plt.scatter(results_frame.Volatility, results_frame.Return, c=results_frame.Sharpe, cmap='viridis', alpha=0.5)
plt.colorbar(label='Sharpe Ratio')

# Highlight the Optimal Portfolio (Red Star)
plt.scatter(max_sharpe_port['Volatility'], max_sharpe_port['Return'], marker='*', color='red', s=300, label='Max Sharpe (Optimal)')

# Highlight the Safest Portfolio (Green Star)
plt.scatter(min_vol_port['Volatility'], min_vol_port['Return'], marker='*', color='green', s=300, label='Min Volatility (Safest)')

plt.title('Efficient Frontier: Risk vs. Return Optimization')
plt.xlabel('Risk (Annualized Volatility)')
plt.ylabel('Expected Annual Return')
plt.legend(labelspacing=1.2)
plt.show()